In [16]:
import pandas as pd
df=pd.read_csv("Stocks_dataset.csv")
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [17]:
df

,Date,Price_VWCE,Price_SP500,Price_MSCI
0,2023-03-01,94.88,4109.31,117.67
1,2023-04-01,95.38,4169.48,119.79
2,2023-05-01,97.58,4179.83,118.60
3,2023-06-01,100.98,4450.38,124.52
4,2023-07-01,103.81,4588.96,128.54
5,2023-08-01,102.69,4507.66,125.70
6,2023-09-01,101.23,4288.05,120.17
7,2023-10-01,97.56,4193.80,117.11
8,2023-11-01,103.13,4567.80,127.78
9,2023-12-01,106.94,4769.83,133.02


Prediction MLP

In [18]:
# Estrai le caratteristiche (prezzo1, prezzo2) e il target (prezzo3)
X = df[['Price_VWCE', 'Price_SP500']].values  # Input (due prezzi)
y = df['Price_MSCI'].values  # Target (prezzo3)

# Converti in tensori PyTorch
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)  # Target deve essere una colonna

In [19]:
# Definisci il modello
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(2, 64)  # 2 input (prezzo1 e prezzo2)
        self.fc2 = nn.Linear(64, 32)  # Un layer nascosto
        self.fc3 = nn.Linear(32, 1)   # Un output (prezzo3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # Funzione di attivazione ReLU
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Crea il modello
model = SimpleModel()

# Funzione di perdita e ottimizzatore
criterion = nn.MSELoss()  # Mean Squared Error per regressione
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [20]:
# Numero di epoche per l'allenamento
epochs = 500

for epoch in range(epochs):
    model.train()

    # Passaggio in avanti
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)

    # Backpropagation e ottimizzazione
    optimizer.zero_grad()  # Azzeriamo i gradienti
    loss.backward()  # Calcoliamo i gradienti
    optimizer.step()  # Aggiorniamo i pesi

    if (epoch + 1) % 100 == 0:  # Ogni 100 epoche stampiamo il valore della loss
        print(f'Epoca [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoca [100/500], Loss: 12.9634
Epoca [200/500], Loss: 9.0356
Epoca [300/500], Loss: 9.0338
Epoca [400/500], Loss: 9.0313
Epoca [500/500], Loss: 9.0286


In [21]:
# Esegui una previsione con nuovi valori di prezzo1 e prezzo2
prezzo1_input = 130.6
prezzo2_input = 5776.25

# Prepara i dati di input per il modello (due prezzi)
input_data = torch.tensor([[prezzo1_input, prezzo2_input]], dtype=torch.float32)

# Fai la previsione
model.eval()  # Modalità valutazione (senza calcolare i gradienti)
predizione = model(input_data)

print(f"Predizione: {predizione.item():.2f}")

Predizione: 158.44


Prediction LSTM

In [24]:


print("DataFrame originale:")
print(df)

# 1. Preprocessing dei dati
# Rimuoviamo la colonna 'Data' (non la usiamo come feature)
df = df.drop('Date', axis=1)

# Normalizziamo i prezzi
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df.values)

# Funzione per creare sequenze di input e target
def create_sequences(data, seq_length=1):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i+seq_length])
        targets.append(data[i+seq_length])
    return np.array(sequences), np.array(targets)

# Creazione delle sequenze
X, y = create_sequences(scaled_data, seq_length=1)

# Conversione in tensori PyTorch
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

# Split dei dati in training e test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# 2. Creazione del modello
class PricePredictionModel(nn.Module):
    def __init__(self):
        super(PricePredictionModel, self).__init__()
        self.lstm = nn.LSTM(input_size=3, hidden_size=64, num_layers=2, batch_first=True)
        self.fc = nn.Linear(64, 3)  # Predici 3 valori (per Prezzo 1, Prezzo 2, Prezzo 3)

    def forward(self, x):
        out, (hn, cn) = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Usa l'output dell'ultimo step della sequenza
        return out

# 3. Definizione dell'ottimizzatore e della funzione di perdita
model = PricePredictionModel()
criterion = nn.MSELoss()  # Mean Squared Error
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. Allenamento del modello
epochs = 100
for epoch in range(epochs):
    model.train()
    
    # Forward pass
    output = model(X_train)
    loss = criterion(output, y_train)
    
    # Backward pass e ottimizzazione
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Ogni 10 epoche, stampiamo il progresso
    if (epoch + 1) % 10 == 0:
        print(f'Epoca {epoch+1}/{epochs}, Loss: {loss.item()}')

# 5. Previsione dei prezzi futuri
model.eval()  # Modalità di valutazione
with torch.no_grad():
    predictions = model(X_test)

# Rescalare i dati di previsione
predicted_prices = scaler.inverse_transform(predictions.numpy())
actual_prices = scaler.inverse_transform(y_test.numpy())

# Visualizzare i risultati
print("\nPrevisioni:", predicted_prices)
print("Valori reali:", actual_prices)



DataFrame originale:
          Date  Price_VWCE  Price_SP500  Price_MSCI
0   2023-03-01       94.88      4109.31      117.67
1   2023-04-01       95.38      4169.48      119.79
2   2023-05-01       97.58      4179.83      118.60
3   2023-06-01      100.98      4450.38      124.52
4   2023-07-01      103.81      4588.96      128.54
5   2023-08-01      102.69      4507.66      125.70
6   2023-09-01      101.23      4288.05      120.17
7   2023-10-01       97.56      4193.80      117.11
8   2023-11-01      103.13      4567.80      127.78
9   2023-12-01      106.94      4769.83      133.02
10  2024-01-01      110.20      4845.65      134.20
11  2024-02-01      114.29      5096.27      140.28
12  2024-03-01      118.34      5254.35      144.91
13  2024-04-01      116.60      5035.69      139.17
14  2024-05-01      117.53      5277.51      145.71
15  2024-06-01      123.10      5460.48      147.49
16  2024-07-01      123.48      5522.30      149.97
17  2024-08-01      123.05      5648.40    